title: NPFL122, Lecture 9
class: title, langtech, cc-by-nc-sa
# Deterministic Policy Gradient, Advanced RL Algorithms

## Milan Straka

### December 10, 2018

---
section: Refresh
# REINFORCE with Baseline

The returns can be arbitrary – better-than-average and worse-than-average
returns cannot be recognized from the absolute value of the return.

Hopefully, we can generalize the policy gradient theorem using a baseline $b(s)$
to
$$∇_→θ J(→θ) ∝ ∑_{s∈𝓢} μ(s) ∑_{a ∈ 𝓐} \big(q_π(s, a) - b(s)\big) ∇_→θ π(a | s; →θ).$$

A good choice for $b(s)$ is $v_π(s)$, which can be shown to minimize variance of
the estimator. Such baseline reminds centering of returns, given that
$v_π(s) = 𝔼_{a ∼ π} q_π(s, a)$. Then, better-than-average returns are positive
and worse-than-average returns are negative.

The resulting value is also called an _advantage function_
$a_π(s, a) ≝ q_π(s, a) - v_π(s)$.

Of course, the $v_π(s)$ baseline can be only approximated. If neural networks
are used to estimate $π(a|s; →θ)$, then some part of the network is usually
shared between the policy and value function estimation, which is trained using
mean square error of the predicted and observed return.

---
# Parallel Advantage Actor Critic

An alternative to independent workers is to train in a synchronous and
centralized way by having the workes to only generate episodes. Such approach
was described in May 2017 by Celemente et al., who named their agent
_parallel advantage actor-critic_ (PAAC).

![w=70%,h=center](../08/paac_framework.pdf)

---
# Continuous Action Space

Until now, the actions were discreet. However, many environments naturally
accept actions from continuous space. We now consider actions which come
from range $[a, b]$ for $a, b ∈ ℝ$, or more generally from a Cartesian product
of several such ranges:
$$∏_i [a_i, b_i].$$

![w=40%,f=right](../08/normal_distribution.pdf)
A simple way how to parametrize the action distribution is to choose them from
the normal distribution.

Given mean $μ$ and variance $σ^2$, probability density function of $𝓝(μ, σ^2)$
is
$$p(x) ≝ \frac{1}{\sqrt{2 π σ^2}} e^{\large-\frac{(x - μ)^2}{2σ^2}}.$$

---
# Continuous Action Space in Gradient Methods

Utilizing continuous action spaces in gradient-based methods is straightforward.
Instead of the $\softmax$ distribution we suitably parametrize the action value,
usually using the normal distribution. Considering only one real-valued action,
we therefore have
$$π(a | s; →θ) ≝ P\Big(a ∼ 𝓝\big(μ(s; →θ), σ(s; →θ)^2\big)\Big),$$
where $μ(s; →θ)$ and $σ(s; →θ)$ are function approximation of mean and standard
deviation of the action distribution.

The mean and standard deviation are usually computed from the shared
representation, with
- the mean being computed as a regular regression (i.e., one output neuron
  without activation);
- the standard variance (which must be positive) being computed again as
  a regression, followed most commonly by either $\exp$ or
  $\operatorname{softplus}$, where $\operatorname{softplus}(x) ≝ \log(1 + e^x)$.

---
# Continuous Action Space in Gradient Methods

During training, we compute $μ(s; →θ)$ and $σ(s; →θ)$ and then sample the action
value (clipping it to $[a, b]$ if required). To compute the loss, we utilize
the probability density function of the normal distribution (and usually also
add the entropy penalty).

```python
  mu = tf.layers.dense(hidden_layer, 1)[:, 0]
  sd = tf.layers.dense(hidden_layer, 1)[:, 0]
  sd = tf.exp(log_sd)   # or sd = tf.nn.softplus(sd)

  normal_dist = tf.distributions.Normal(mu, sd)

  # Loss computed as - log π(a|s) - entropy_regularization
  loss = - normal_dist.log_prob(self.actions) * self.returns \
         - args.entropy_regularization * normal_dist.entropy()
```

---
section: DPG
# Deterministic Policy Gradient Theorem

Combining continuous actions and Deep Q Networks is not straightforward.
In order to do so, we need a different variant of the policy gradient theorem.


Recall that in policy gradient theorem,
$$∇_→θ J(→θ) ∝ ∑_{s∈𝓢} μ(s) ∑_{a ∈ 𝓐} q_π(s, a) ∇_→θ π(a | s; →θ).$$


## Deterministic Policy Gradient Theorem
Assume that the policy $π(s; →θ)$ is deterministic and computes
an action $a∈ℝ$. Then under several assumptions about continuousness, the
following holds:
$$∇_→θ J(→θ) ∝ 𝔼_{s∼μ(s)} \Big[∇_→θ π(s; →θ) ∇_a q_π(s, a)\big|_{a=π(s;→θ)}\Big].$$

The theorem was first proven in the paper Deterministic Policy Gradient Algorithms
by David Silver et al.

---
# Deterministic Policy Gradient Theorem – Proof

The proof is very similar to the original (stochastic) policy gradient theorem.
We assume that $p(s' | s, a), ∇_a p(s' | s, a), r(s, a), ∇_a r(s, a), π(s; →θ), ∇_→θ π(s; →θ)$
are continuous in all params.


$\displaystyle ∇_→θ v_π(s) = ∇_→θ q_π(s, π(s; →θ))$


$\displaystyle \phantom{∇_→θ v_π(s)} = ∇_→θ\Big(r\big(s, π(s; →θ)\big) + γ ∫_{s'} p\big(s' | s, π(s; →θ)\big) v_π(s') \d s'\Big)$


$\displaystyle \phantom{∇_→θ v_π(s)} = ∇_→θ π(s; →θ) ∇_a r(s, a) \big|_{a=π(s; →θ)} + γ ∇_→θ ∫_{s'} p\big(s' | s, π(s; →θ)\big) v_π(s') \d s'$


$\displaystyle \phantom{∇_→θ v_π(s)} = ∇_→θ π(s; →θ) ∇_a \Big( r(s, a) \big|_{a=π(s; →θ)} + γ ∫_{s'} p\big(s' | s, a)\big) v_π(s') \d s' \Big) \\
                    \qquad\qquad\qquad + γ ∫_{s'} p\big(s' | s, π(s; →θ)\big) ∇_→θ v_π(s') \d s'$


$\displaystyle \phantom{∇_→θ v_π(s)} = ∇_→θ π(s; →θ) ∇_a q_π(s, a)\big|_{a=π(s; →θ)} + γ ∫_{s'} p\big(s' | s, π(s; →θ)\big) ∇_→θ v_π(s') \d s'$


Similarly to the gradient theorem, we finish the proof by continually expanding $∇_→θ v_π(s')$.

---
section: DDPG
# Deep Deterministic Policy Gradients

Note that the formulation of deterministic policy gradient theorem allows an
off-policy algorithm, because the loss functions no longer depends on actions
(similarly to how expected Sarsa is also an off-policy algorithm).


We therefore train function approximation for both $π(s; →θ)$ and $q(s, a; →θ)$,
training $q(s, a; →θ)$ using a deterministic variant of the Bellman equation:
$$q(S_t, A_t; →θ) = 𝔼_{R_{t+1}, S_{t+1}} \big[R_{t+1} + γ q(S_{t+1}, π(S_{t+1}; →θ))\big]$$
and $π(s; →θ)$ according to the deterministic policy gradient theorem.


The algorithm was first described in the paper Continuous Control with Deep Reinforcement Learning
by Timothy P. Lillicrap et al. (2015).

The authors utilize a replay buffer, a target network (updated by exponential
moving average with $τ=0.001$), batch normalization for CNNs, and perform
exploration by adding a normal-distributed noise to predicted actions.
Training is performed by Adam with learning rates of 1e-4 and 1e-3 for the
policy and critic network, respectively.

---
# Deep Deterministic Policy Gradients

![w=65%,h=center](ddpg.pdf)

---
# Deep Deterministic Policy Gradients

![w=100%](ddpg_ablation.pdf)

---
# Deep Deterministic Policy Gradients

Results using low-dimensional (_lowd_) version of the environment, pixel representation
(_pix_) and DPG reference (_cntrl_).

![w=57%,h=center](ddpg_results.pdf)

---
section: NPG
# Natural Policy Gradient

The following approach has been introduced by Kakade (2002).


Using policy gradient theorem, we are able to compute $∇ v_π$. Normally, we
update the parameters by using directly this gradient. This choice is justified
by the fact that a vector $→d$ which maximizes $v_π(s; →θ + →d)$ under
the constraint that $|→d|^2$ is bounded by a small constant is exactly
the gradient $∇ v_π$.


Normally, the length $|→d|^2$ is computed using Euclidean metric. But in general,
any metric could be used. Representing a metric using a positive-definite matrix
$⇉G$ (identity matrix for Euclidean metric), we can compute the distance as
$|→d|^2 = ∑_{ij} G_{ij} d_i d_j = →d^T ⇉G →d$. The steepest ascent direction is
then given by $⇉G^{-1} ∇ v_π$.


Note that when $⇉G$ is the Hessian $⇉H v_π$, the above process is exactly
Newton's method.

---
# Natural Policy Gradient

![w=100%,v=middle](npg.pdf)

---
# Natural Policy Gradient

A suitable choice for the metric is _Fisher information matrix_ defined as
$$F_s(→θ) ≝ 𝔼_{π(a | s; →θ)} \left[\frac{∂ \log π(a | s; →θ)}{∂ →θ_i} \frac{∂ \log π(a | s; →θ)}{∂ →θ_j} \right]
\color{gray} = 𝔼[∇ π(a | s; →θ)] 𝔼[∇ π(a | s; →θ)]^T.$$


It can be shown that the Fisher information metric is the only Riemannian metric
(up to rescaling) invariant to change of parameters under sufficient statistic.


Recall Kullback-Leibler distance (or relative entropy) defined as
$$D_\textrm{KL}(→p || →q) ≝ ∑_i p_i \log \frac{p_i}{q_i} \color{gray} = H(p, q) - H(p).$$


The Fisher information matrix is also a Hessian of the
$D_\textrm{KL}(π(a | s; →θ) || π(a | s; →θ')$:
$$F_s(→θ) = \frac{∂^2}{∂θ_i' ∂θ_j'} D_\textrm{KL}(π(a | s; →θ) || π(a | s; →θ')\Big|_{→θ' = →θ}.$$

---
# Natural Policy Gradient

Using the metric
$$F(→θ) = 𝔼_{s ∼ μ_→θ} F_s(→θ)$$
we want to update the parameters using $→d_F ≝ F(→θ)^{-1} ∇ v_π$.


An interesting property of using the $→d_F$ to update the parameters is that
- updating $→θ$ using $∇ v_π$ will choose an arbitrary _better_ action in state
  $s$;

- updating $→θ$ using $F(→θ)^{-1} ∇ v_π$ chooses the _best_ action (maximizing
  expected return), similarly to tabular greedy policy improvement.


However, computing $→d_F$ in a straightforward way is too costly.

---
# Truncated Natural Policy Gradient

Duan et al. (2016) in paper _Benchmarking Deep Reinforcement Learning for
Continuous Control_ propose a modification to the NPG to efficiently compute
$→d_F$.


Following Schulman et al. (2015), they suggest to use _conjugate gradient
algorithm_, which can solve a system of linear equations $⇉A→x = →b$
in an iterative manner, by using $⇉A$ only to compute products $⇉A→v$ for
a suitable $→v$.


Therefore, $→d_F$ is found as a solution of
$$F(→θ)→d_F = ∇ v_π$$
and using only 10 iterations of the algorithm seem to suffice according to the
experiments.


Furthermore, Duan et al. suggest to use a specific learning rate suggested by
Peters et al (2008) of
$$\frac{α}{\sqrt{(∇ v_π)^T F(→θ)^{-1} ∇ v_π}}.$$

---
section: TRPO
# Trust Region Policy Optimization

Schulman et al. in 2015 wrote an influential paper introducing TRPO as an
improved variant of NPG.


Considering two policies $π, π̃$, we can write
$$v_π̃ = v_π + 𝔼_{s ∼ μ(π̃)} 𝔼_{a ∼ π̃(a | s)} a_π(a | s),$$
where $a_π(a | s)$ is the advantage function $q_π(a | s) - v_π(s)$ and
$μ(π̃)$ is the on-policy distribution of the policy $π̃$.


Analogously to policy improvement, we see that if $a_π(a | s) ≥0$, policy
$π̃$ performance increases (or stays the same if the advantages are zero
everywhere).


However, sampling states $s ∼ μ(π̃)$ is costly. Therefore, we instead
consider
$$L_π(π̃) = v_π + 𝔼_{s ∼ μ(π)} 𝔼_{a ∼ π̃(a | s)} a_π(a | s).$$

---
# Trust Region Policy Optimization
$$L_π(π̃) = v_π + 𝔼_{s ∼ μ(π)} 𝔼_{a ∼ π̃(a | s)} a_π(a | s)$$

It can be shown that for parametrized $π(a | s; →θ)$ the $L_π(π̃)$ matches
$v_{π̃}$ to the first order.


Schulman et al. additionally proves that if we denote
$α = D_\textrm{KL}^\textrm{max}(π_\textrm{old} || π_\textrm{new})
   = \max_s D_\textrm{KL}\big(π_\textrm{old}(⋅|s) || π_\textrm{new}(⋅|s)\big)$, then
$$v_{π_\textrm{new}} ≥ L_{π_\textrm{old}}(π_\textrm{new}) - \frac{4εγ}{(1-γ)^2}α\textrm{where}ε = \max_{s, a} |a_π(s, a)|.$$


Therefore, TRPO minimizes $L_{π_{→θ_0}}(π_→θ)$ subject to
$D_\textrm{KL}^{→θ_0}(π_{→θ_0} || π_→θ) < δ$, where
- $D_\textrm{KL}^{→θ_0}(π_{→θ_0} || π_→θ) = 𝔼_{s ∼ μ(π_{→θ_0})} [D_\textrm{KL}\big(π_\textrm{old}(⋅|s) || π_\textrm{new}(⋅|s)\big)]$
  is used instead of $D_\textrm{KL}^\textrm{max}$ for performance reasons;

- $δ$ is a constant found empirically, as the one implied by the above equation
  is too small;

- importance sampling is used to account for sampling actions from $π$.

---
# Trust Region Policy Optimization

$$\textrm{minimize}~~L_{π_{→θ_0}}(π_→θ)~~\textrm{subject to}~~D_\textrm{KL}^{→θ_0}(π_{→θ_0} || π_→θ) < δ$$

The parameters are updated using $→d_F = F(→θ)^{-1} ∇ L_{π_{→θ_0}}(π_→θ)$, utilizing the
conjugate gradient algorithm as described earlier for TNPG (note that the
algorithm was designed originally for TRPO and only later employed for TNPG).


To guarantee improvement and respect the $D_\textrm{KL}$ constraint, a line
search is in fact performed. We start by the learning rate of
$\sqrt{δ/(→d_F^T F(→θ)^{-1} →d_F)}$ and shrink it exponentially until
the constraint is satistifed and the objective improves.

---
# Trust Region Policy Optimization

![w=30%,h=center](rllib_tasks.pdf)

![w=100%](rllib_results.pdf)

---
section: PPO
# Proximal Policy Optimization

A simplification of TRPO which can be implemented using a few lines of code.

Let $r_t(→θ) ≝ \frac{π(A_t|S_t; →θ)}{π(A_t|S_t; →θ_\textrm{old})}$. PPO
minimizes the objective
$$L^\textrm{CLIP}(→θ) ≝ 𝔼_t\Big[\min\big(r_t(→θ) Â_t, \operatorname{clip}(r_t(→θ), 1-ε, 1+ε) Â_t)\big)\Big].$$

Such $L^\textrm{CLIP}(→θ)$ is a lower (pessimistic) bound.

![w=60%,h=center](ppo_clipping.pdf)

---
# Proximal Policy Optimization

The advantages $Â_t$ are additionally estimated using _generalized
advantage estimation_. Instead of the usual
$Â_t ≝ ∑_{i=0}^{T-t-1} γ^i R_{t+1+i} + γ^{T-t} V(S_T) - V(S_t)$
the authors employ
$$Â_t ≝ ∑_{i=0}^{T-t-1} (γλ)^i δ_{t+i},$$
where $δ_t = R_{t+1} + γV(S_{t+1}) - V(S_t)$.

![w=80%,h=center](ppo_algorithm.pdf)

---
# Proximal Policy Optimization

![w=100%,v=middle](ppo_results.pdf)

---
section: SAC
# Soft Actor Critic

The paper Soft Actor-Critic: Off-Policy Maximum Entropy Deep Reinforcement
Learning with a Stochastic Actor by Tuomas Haarnoja et al. introduces
a different off-policy algorithm for continuous action space.


The general idea is to introduce entropy directly in the value function we want
to maximize.

---
# Soft Actor Critic
![w=60%,h=center](sac_algorithm.pdf)

---
# Soft Actor Critic
![w=90%](sac_results.pdf)
